In [1]:
import pandas as pd
import numpy as np
import csv
import re
import networkx as nx
import matplotlib.pyplot as plt
from math import log # IDF 계산을 위해
from konlpy.tag import Okt
from konlpy.tag import Mecab

In [2]:
df = pd.read_csv('./bangdream.csv')
df.values

array([['RT @seol_S2_ah: #뱅드림_천사님\n\n뱅드림 천사님을 친드로 모십니다!\n자세한 건 세번째 사진 읽어주시고, 아래 표 작성하신 후 DM 찔러주심 감사하겠습니다.\n\n트친분들 인알 부탁드려요🙏🏻😊 https://t.co/rTxUogjDeD'],
       ['RT @bang_goods111: 뱅드림 사토 마스키 마스킹 굿즈 처분 판매합니다\n\n- 사진 별로 묶음으로 일괄 판매\n- 개별 가격(소분용)은 디엠으로 문의주세요\n- 네모표시: 미개봉 or 미전시품\n-일괄 구매 시 600,000\n- 슈발츠: 10장,…'],
       ['RT @Wedny_bangdr: #뱅드림_트친소 #트친소\n\n뱅드림 로젤리아, 이마이 리사 좋아합니다 ♡ 굿즈 쫌쫌따리 모으고 시간 나면 그림도 그리고 오너캐 얘기도 해요! 그림은 라톤님 ♪ https://t.co/qudOyBR6nr'],
       ['뱅드림 필름라이브 필라2 특전 색지 2주차 모르포니카 키리가야 토우코 반택포 1.0에 판매합니다! https://t.co/SFUEuHyJ1J'],
       ['RT @110110_kki: 나 어릴 때 그림 연습 그걸로함\n앙스타뱅드림방법\n\n남캐 잘그리고싶으면 앙스타 일러 내그림체로 그리기 200번\n여캐 잘그리고싶으면 뱅드림 일러 내그림체로 그리기 200번\n\n그래서 진짜 했냐고 물어보신다면\n둘다 합쳐서 13번…'],
       ['RT @qlqlaas: 뱅드림 쿠라타 마시로 굿즈 구합니다!\n아크릴과 지류 위주로 구하고 있어요\n디엠 주세요🙏🏻!!'],
       ['RT @aroni_lo: #rt\n#천사님_와주세요 \n#뱅드림_천사님 \n#아야 천사님 \n#천사님모셔요 \n뱅드림 장르의 마루야마 아야 천사님을 모십니다!!\n아래 글 읽고 와주세요!! https://t.co/FrRwL1b64B'],
       ['RT @ddongdohoon: 뱅드림 츄츄 타마데 치유\n라스 위주 굿즈 위주로 팝니다!\n일괄 &gt; 다수 &

In [3]:
df['text'][0:5]

0    RT @seol_S2_ah: #뱅드림_천사님\n\n뱅드림 천사님을 친드로 모십니다!...
1    RT @bang_goods111: 뱅드림 사토 마스키 마스킹 굿즈 처분 판매합니다\...
2    RT @Wedny_bangdr: #뱅드림_트친소 #트친소\n\n뱅드림 로젤리아, 이...
3    뱅드림 필름라이브 필라2 특전 색지 2주차 모르포니카 키리가야 토우코 반택포 1.0...
4    RT @110110_kki: 나 어릴 때 그림 연습 그걸로함\n앙스타뱅드림방법\n\...
Name: text, dtype: object

In [4]:
def preprocessing(text):
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',text)
    text = re.sub('[\s]+', ' ', text)
    text = text.lower()
    return text

In [5]:
df['text'] = df['text'].map(preprocessing)

In [6]:
df['text'][0]

' 뱅드림천사님 뱅드림 천사님을 친드로 모십니다 자세한 건 세번째 사진 읽어주시고 아래 표 작성하신 후 찔러주심 감사하겠습니다 트친분들 인알 부탁드려요 '

In [7]:
from konlpy.tag import Okt
Okt()

In [8]:
def okt_clean(text):
    okt_pos = Okt().pos(text, stem =True)
    clean_text = []
    for op in okt_pos:
        if op[1] not in ["Josa", "Eomi", "Punctuation"]:
            clean_text.append(op[0])
    return " ".join(clean_text)

In [9]:
from tqdm import tqdm
tqdm.pandas()

df['text'] = df['text'].progress_map(okt_clean)

100%|██████████████████████████████████████| 1000/1000 [00:03<00:00, 312.65it/s]


In [10]:
df['text'][0:5]

0    뱅 드림 천사 님 뱅 드림 천사 님 친드 모시다 자세하다 건 세번 째 사진 읽다 아...
1    뱅 드림 사토 마 스키 마스 킹 굿 즈 처분 판매 하다 사진 별로 묶음 일괄 판매 ...
2    뱅 드림 트친소 트친소 뱅 드림 로젤리아 이마이 리사 좋아하다 굿 즈 쫌쫌 따리 모...
3    뱅 드림 필름 라이브 필라 특전 색 주차 모 르포 카 키 리가 토우코 반택 포 판매 하다
4    나 어리다 때 그림 연습 그 걸 함 앙 스타 뱅 드림 방법 남캐 잘 그리다 앙 스타...
Name: text, dtype: object

In [11]:
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['옾챗','있다','ㅠ','합니다', '하는', '할', '하고', '한다', '그리고', '트친소', '등', '그', '입니다', '및', '제', '더']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [12]:
df['text'] = df['text'].progress_map(remove_stopwords)

100%|████████████████████████████████████| 1000/1000 [00:00<00:00, 63112.10it/s]


In [13]:
df['text'][0:5]

0    뱅 드림 천사 님 뱅 드림 천사 님 친드 모시다 자세하다 건 세번 째 사진 읽다 아...
1    뱅 드림 사토 마 스키 마스 킹 굿 즈 처분 판매 하다 사진 별로 묶음 일괄 판매 ...
2    뱅 드림 뱅 드림 로젤리아 이마이 리사 좋아하다 굿 즈 쫌쫌 따리 모으다 시간 나다...
3    뱅 드림 필름 라이브 필라 특전 색 주차 모 르포 카 키 리가 토우코 반택 포 판매 하다
4    나 어리다 때 그림 연습 걸 함 앙 스타 뱅 드림 방법 남캐 잘 그리다 앙 스타 이...
Name: text, dtype: object

In [14]:
df

,text
0,뱅 드림 천사 님 뱅 드림 천사 님 친드 모시다 자세하다 건 세번 째 사진 읽다 아...
1,뱅 드림 사토 마 스키 마스 킹 굿 즈 처분 판매 하다 사진 별로 묶음 일괄 판매 ...
2,뱅 드림 뱅 드림 로젤리아 이마이 리사 좋아하다 굿 즈 쫌쫌 따리 모으다 시간 나다...
3,뱅 드림 필름 라이브 필라 특전 색 주차 모 르포 카 키 리가 토우코 반택 포 판매 하다
4,나 어리다 때 그림 연습 걸 함 앙 스타 뱅 드림 방법 남캐 잘 그리다 앙 스타 이...
...,...
995,뱅 드림 진행 계 깔다 돌다 판매 양 도합 니 계좌 문상 받다 계좌 문상 키라 페스...
996,뱅 드림 합동 라이브 공연 티켓 하 구미 운 밉다 파다 교환 환불 안되다
997,뱅 드림 헬로 해피 헬햎 세타 카오루 포토 카드 백스테 스티커 판매 양 도합 니 백...
998,뱅 드림 합동 라이브 공연 티켓 하 구미 운 밉다 파다 교환 환불 안되다


In [15]:
my_data_array = np.array(df)
print(my_data_array)

[['뱅 드림 천사 님 뱅 드림 천사 님 친드 모시다 자세하다 건 세번 째 사진 읽다 아래 표 작성 하다 후 찌르다 주심 감사하다 트친 분들 인알 부탁드리다']
 ['뱅 드림 사토 마 스키 마스 킹 굿 즈 처분 판매 하다 사진 별로 묶음 일괄 판매 개별 가격 소 분 용다 디엠 문의 줄다 네 모 표시 미개 봉 미전 시품 일괄 구매 시 슈발츠 장']
 ['뱅 드림 뱅 드림 로젤리아 이마이 리사 좋아하다 굿 즈 쫌쫌 따리 모으다 시간 나다 그림 오너캐 얘기 해 그림 톤 님']
 ['뱅 드림 필름 라이브 필라 특전 색 주차 모 르포 카 키 리가 토우코 반택 포 판매 하다']
 ['나 어리다 때 그림 연습 걸 함 앙 스타 뱅 드림 방법 남캐 잘 그리다 앙 스타 이르다 내 그림 체 그리기 번 여캐 잘 그리다 뱅 드림 이르다 내 그림 체 그리기 번 그래서 진짜 하다 물어보다 두다 합치다 번']
 ['뱅 드림 쿠라 타 말다 굿 즈 구합 니 아크릴 지류 위주 구 디엠 줄다']
 ['천사 님 줄다 뱅 드림 천사 님 아야 천사 님 천사 님 모시다 뱅 드림 장르 마루 야마 아야 천사 님 모시다 아래 글 읽다 오다']
 ['뱅 드림 츄츄 타마 데 치유 라스 위주 굿 즈 위주 파다 일괄 다수 개별 순 우대 하다 하자 예민하다 신분 들 삼가다 해주다 자세하다 디엠 물어보다']
 ['최소 이상 거래 하다 사진 가격 적히다 전체 적 개봉 후 전시 해 두다 하다 상품 자세하다 하자 확인 하다 디엠 부탁드리다 자세하다 사항 타래 적다 두다 레이 뱅']
 ['뱅 드림 반 도리 포핀 파티 포 피파 하나조노 타 포핀 드림 앨범 특전 포토 카드 각 판매 양 도합 니 일괄 구매 시 반택 비 부담 하다']
 ['뱅 드림 굿 즈 마스 킹 모리 체크 모리 척 누 이 구합 니 문의 디엠 부탁드리다']
 ['뱅 드림 이 치가 아리사 넨도 카페 아크릴 하나조노 타 에이치 가야 아리사 필름 라이브 색 오다 카미 이브 부쿠부 스트랩 기모노 여름 아크릴 판매 하다 일괄 시 운포 문의']
 ['뱅 드림 반 도리 깔다 돌

In [16]:
pd.DataFrame(my_data_array).to_csv('bangdream_jun.csv')    